In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("July_2021_New_202208222035.csv")

In [3]:
df.head(4)

,actorId,verb,type,activity,ancestors,duration,completion,Is Manager,Time Zone,timestamp,other,contained,rating,response,row_number
0,account|https://coles.clearlrs.com|3fa55218f73...,http://adlnet.gov/expapi/verbs/registered,http://adlnet.gov/expapi/activities/lesson,https://coles.clearlrs.com/activities/_6Ac3BGi...,[{'activity': 'https://coles.clearlrs.com/acti...,10070,NaN,Y,Australia/Melbourne,2021-07-01T00:00:00.246Z,[{'activity': 'https://coles.hub.clearlrs.com'...,NaN,NaN,NaN,1
1,account|https://coles.clearlrs.com|9268e85c723...,http://adlnet.gov/expapi/verbs/completed,NaN,https://coles.clearlrs.com/activities/assets/0...,[{'activity': 'https://coles.clearlrs.com/acti...,401,1.0,N,Australia/Brisbane,2021-07-01T00:00:02.599Z,NaN,NaN,NaN,NaN,1
2,account|https://coles.clearlrs.com|7b027350-6f...,http://adlnet.gov/expapi/verbs/initialized,NaN,https://coles.hub.clearlrs.com,[{'activity': 'https://coles.hub.clearlrs.com'...,0,0.0,N,Australia/Melbourne,2021-07-01T00:00:04.375Z,NaN,NaN,NaN,NaN,1
3,account|https://coles.clearlrs.com|7b027350-6f...,http://adlnet.gov/expapi/verbs/experienced,NaN,https://coles.hub.clearlrs.com/learning,[{'activity': 'https://coles.hub.clearlrs.com'...,38641,0.0,N,Australia/Melbourne,2021-07-01T00:00:04.395Z,NaN,NaN,NaN,NaN,2


In [4]:
def to_dict(col):
    index_list = list(range(len(col)))
    res_dct = dict(zip(col, index_list))
    return res_dct

In [5]:
# verb: keep the last word
list_verb = df['verb'].tolist()
list_verb = [ i.split('/')[-1] for i in list_verb]
df['verb'] = list_verb

In [6]:
# actorid: convert to dict
list_actorid = df['actorId'].unique()
dict_actorid = to_dict(list_actorid)
temp_list = [dict_actorid[i] for i in df['actorId']]
df['actorId'] = temp_list

In [7]:
# type: keep last word
list_type = df['type'].tolist()
list_type = [ str(i).split('/')[-1] for i in list_type]
for i in range(0,len(list_type)):
    if list_type[i] == 'nan':
        list_type[i] = pd.NA
df['type'] = list_type

In [8]:
#Is manager: YN to 10
df.loc[(df["Is Manager"] == "Y"), "Is Manager"] = 1
df.loc[(df["Is Manager"] == "N"), "Is Manager"] = 0

In [9]:
#time zone: convert to dict
list_timezone = df['Time Zone'].unique()
dict_timezone = to_dict(list_timezone)
temp_list = [dict_timezone[i] for i in df['Time Zone']]
df['Time Zone'] = temp_list

In [10]:
def manual_label(i):
    if i == 'https://coles.hub.clearlrs.com':
        i = "home"
        pass
    if "activities" in i:
        i = "activities"
        pass
    if "course_id" in i:
        i = 'course_id'
        pass
    if "notifications" in i:
        i = 'notifications'
        pass
    if "course/" in i:
        i = "learningseat_course"
        pass
    if "colesplay" in i:
        i = "colesplay"
        pass
    i = i.split('/')[-1] 
    
    return i

In [11]:
# activity: manually label
list_activity = df['activity']
list_activity = np.array(list_activity)
temp_list = []
for i in list_activity:
    res = manual_label(i)
    temp_list.append(res)

df['activity'] = temp_list

In [12]:
# ancestors,[each activities]
list_ancestors = df['ancestors']
list_ancestors = np.array(list_ancestors)
temp_list = []
for i in list_ancestors:
    lines = i[1:-1].split(",")
    temp_list.append([manual_label(lines[i][14:-1]) for i in range(len(lines)) if i % 2 == 0])
df['ancestors'] = temp_list

In [13]:
# other: use manaul label
list_other = df.other
temp_list =[]
for i in range(len(list_other)):
    if str(list_other[i]) == '<NA>' or str(list_other[i]) == 'nan':
        temp_list.append(pd.NA)
    else:
        temp_list.append(manual_label(list_other[i][15:-15]))
        
df['other'] = temp_list

In [14]:
# contained, [search actorid dict, verb last word]
list_contain = df.contained
temp_list = []
for i in range(0,len(list_contain)):
    if str(list_contain[i]) == '<NA>' or str(list_contain[i]) == 'nan':
        temp_list.append(pd.NA)
    else:
        target_id = list_contain[i].split(',')[0][13:-1]
        result = []
        try:
            result.append(dict_actorid[target_id])
        except KeyError:
            result.append('unrecorded')
        
        contained_verb = list_contain[i].split(',')[1][10:-2].split('/')[-1]
        result.append(contained_verb)
        temp_list.append(result)
df.contained = temp_list

In [15]:
df.to_csv('cleaned.csv')